In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [14]:
df = pd.read_csv('raw_logs.csv',header=0).groupby('session')
for session, sdf in df:
    timeStr = datetime.fromtimestamp(session/1000.0).strftime('%Y-%m-%d %H:%M:%S')
    print('session {0} ({1}), {2} records'.format(session, timeStr, len(sdf.index)))
    
    # Sort by timestamp get the final average MPG for every trip (kff1205)
    sdf = sdf.sort_values('time')
    
    # Remove invalid values
    sdf = sdf[sdf.k43 != 0]
    
    # Print trip start and end timestamp
    tsList = sdf.time.tolist()
    sts = tsList[0]
    ets = tsList[-1]
    stsStr = datetime.fromtimestamp(sts/1000.0).strftime('%Y-%m-%d %H:%M:%S')
    etsStr = datetime.fromtimestamp(ets/1000.0).strftime('%Y-%m-%d %H:%M:%S')
    print("\t|- Trip start {0}, end {1}".format(stsStr, etsStr))
    # Print trip total length
    diffTs = ets - sts
    print("\t|- Trip length {0} seconds".format(diffTs/1000.0))
    #

    print("\n")
    # Key values
    ## k11     =  Throttle Position(Manifold)
    ## kc      =  Engine RPM
    ## kff1271 =  Fuel used (trip)
    ## kff1204 =  Trip Distance
    ## kff1205 =  Trip average MPG

session 1509673999804 (2017-11-02 20:53:19), 1678 records
	|- Trip start 2017-11-02 20:59:29, end 2017-11-02 21:27:23
	|- Trip length 1673.965 seconds


session 1509715520133 (2017-11-03 08:25:20), 493 records
	|- Trip start 2017-11-03 08:42:34, end 2017-11-03 08:50:43
	|- Trip length 488.986 seconds


session 1509814879698 (2017-11-04 12:01:19), 468 records
	|- Trip start 2017-11-04 12:05:15, end 2017-11-04 12:12:59
	|- Trip length 463.987 seconds


session 1510237809601 (2017-11-09 08:30:09), 2361 records
	|- Trip start 2017-11-09 08:36:18, end 2017-11-09 09:15:35
	|- Trip length 2356.962 seconds




In [15]:
sdf.sort_values('time').keys()

Index(['v', 'session', 'id', 'time', 'kff1005', 'kff1006', 'kff1001',
       'kff1007', 'k4', 'k2f', 'k11', 'k5', 'kc', 'kd', 'kf', 'kff1226',
       'kff1220', 'kff1221', 'k46', 'eml', 'k43', 'kff1271', 'k1f', 'kff1205',
       'kff1204', 'kff1266', 'kff1270'],
      dtype='object')

[1509674369109,
 1509674370073,
 1509674371073,
 1509674372073,
 1509674373073,
 1509674374073,
 1509674375073,
 1509674376073,
 1509674377073,
 1509674378073,
 1509674379074,
 1509674380074,
 1509674381074,
 1509674382073,
 1509674383073,
 1509674384074,
 1509674385074,
 1509674386074,
 1509674387074,
 1509674388074,
 1509674389073,
 1509674390074,
 1509674391074,
 1509674392073,
 1509674393073,
 1509674394073,
 1509674395074,
 1509674396073,
 1509674397077,
 1509674398074,
 1509674399075,
 1509674400075,
 1509674401074,
 1509674402075,
 1509674403074,
 1509674404073,
 1509674405074,
 1509674406074,
 1509674407073,
 1509674408074,
 1509674409074,
 1509674410074,
 1509674411074,
 1509674412074,
 1509674413074,
 1509674414074,
 1509674415074,
 1509674416074,
 1509674417074,
 1509674418075,
 1509674419073,
 1509674420075,
 1509674421075,
 1509674422079,
 1509674423074,
 1509674424073,
 1509674425074,
 1509674426074,
 1509674427074,
 1509674428074,
 1509674429075,
 1509674430073,
 1509674